In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import os
import nltk
from textblob import TextBlob
import openpyxl
from nltk.sentiment import SentimentIntensityAnalyzer


In [2]:
#Importing the Input.xlsx file
df = pd.read_excel('Input.xlsx')

In [10]:
#this piece of code is mainly to extract the Title and Text from the url but we will find that 
#it is not fully processed

def extract_text(url):
    """
    Extracting article title and text from a URL using BeautifulSoup.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Assuming 'title' and 'article' are HTML tags containing the title and text
        title = soup.find('title').get_text() if soup.find('title') else ''
        article = soup.find('article').get_text() if soup.find('article') else ''

        return title, article

    except Exception as e:
        print(f"Error extracting text from {url}: {str(e)}")
        return '', ''

def clean_title_for_filename(title):
    """
    Replace invalid characters in the title with underscores.
    """
    return re.sub(r'[\/:*?"<>|]', '_', title)

def main():
    df = pd.read_excel('Input.xlsx')

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Extract URL_ID and URL
        url_id = str(row['URL_ID'])
        url = str(row['URL'])

        # Print progress information
        print(f"Processing Article {url_id}: {url}")

        # Extract text from the URL
        title, article_text = extract_text(url)

        # Clean the title for creating a filename
        cleaned_title = clean_title_for_filename(title)

        # Create a text file for each article and save the extracted text
        output_file_path = f"{url_id}_{cleaned_title[:20]}.txt"  # Limit title length in filename for practicality
        try:
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.write(f"Title: {title}\n\n")
                file.write(f"Text: {article_text}")
        except FileNotFoundError as file_error:
            print(f"Error creating file for Article {url_id}: {str(file_error)}")

if __name__ == "__main__":
    main()


Processing Article blackassign0001: https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/
Processing Article blackassign0002: https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/
Processing Article blackassign0003: https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/
Processing Article blackassign0004: https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/
Processing Article blackassign0005: https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/
Processing Article blackassign0006: https://insights.blackcoffer.com/the-rise-of-the-ott-platform-and-its-impact-on-the-entertainment-industry-by-2040/
Processing Article blackassign0007: https://insights.blackcoffer.com/rise-of-cyber-crime-an

In [21]:
#file was not correct in extracting the title and text thus another code was written to 
#remove excess number of lines before the main text 


def remove_lines_after_text(file_path):
    # Read the content of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the line index containing 'Text :'
    text_line_index = -1
    for i, line in enumerate(lines):
        if 'Text:' in line:
            text_line_index = i
            break

    # Remove the next 10 lines after 'Text :'
    if text_line_index != -1:
        del lines[text_line_index + 1:text_line_index + 15]

    # Write the modified content back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path):
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")
            
            # Remove lines after 'Text :' and save the updated content
            remove_lines_after_text(file_path)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing text files
    folder_path = 'C:/Users/Dell/'

    # Process files in the folder
    process_files_in_folder(folder_path)


Processing file: C:/Users/Dell/blackcofferr/blackassign0001_Rising IT cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0002_Rising IT Cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0003_Internet Demand's Ev.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0004_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0005_OTT platform and its.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0006_The rise of the OTT .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0007_Rise of Cyber Crime .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0008_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0009_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0010_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0011_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0012_Rise 

In [17]:
#this one is similar to the above one only minor change to accomodate other problem which came 
#during extracting the text like extra articles title and all thus removing them in this

def remove_lines_after_text(file_path):
    # Read the content of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the line index containing 'Text :'
    text_line_index = -1
    for i, line in enumerate(lines):
        if 'RELATED ARTICLESMORE FROM AUTHOR' in line:
            text_line_index = i
            break

    # Remove the next 10 lines after 'Text :'
    if text_line_index != -1:
        del lines[text_line_index + 1:]

    # Write the modified content back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path):
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")
            
            # Remove lines after 'Text :' and save the updated content
            remove_lines_after_text(file_path)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing text files
    folder_path = 'C:/Users/Dell/'

    # Process files in the folder
    process_files_in_folder(folder_path)


Processing file: C:/Users/Dell/blackcofferr/blackassign0001_Rising IT cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0002_Rising IT Cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0003_Internet Demand's Ev.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0004_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0005_OTT platform and its.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0006_The rise of the OTT .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0007_Rise of Cyber Crime .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0008_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0009_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0010_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0011_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0012_Rise 

In [18]:

# this one is also to reove certain parts of the text which are not part of main text
#here minor changes has been done to accomodate new changes here just becuse of laziness the
#the code is copy pasted from above

def remove_specific_text(file_path, text_to_remove):
    # Read the content of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove the specified text from each line
    lines = [line.replace(text_to_remove, '') for line in lines]

    # Write the modified content back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path, text_to_remove):
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")
            
            # Remove specific text and save the updated content
            remove_specific_text(file_path, text_to_remove)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing text files
    folder_path = 'C:/Users/Dell/'
    
    # Replace '-  Insights' with the specific text you want to remove
    text_to_remove = '-  Insights'

    # Process files in the folder
    process_files_in_folder(folder_path, text_to_remove)


Processing file: C:/Users/Dell/blackcofferr/blackassign0001_Rising IT cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0002_Rising IT Cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0003_Internet Demand's Ev.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0004_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0005_OTT platform and its.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0006_The rise of the OTT .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0007_Rise of Cyber Crime .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0008_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0009_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0010_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0011_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0012_Rise 

In [19]:
#this will finally remove the left behind(which is not part of main content to removed from the
#text

def remove_text_and_following(file_path, text_to_remove):
    # Read the content of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the line index containing the specified text
    text_line_index = -1
    for i, line in enumerate(lines):
        if text_to_remove in line:
            text_line_index = i
            break

# Remove the line containing the specified text and any text following it
    if text_line_index != -1:
        del lines[text_line_index:]

# Writes the modified content back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path, text_to_remove):
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")
            
            
            remove_text_and_following(file_path, text_to_remove)

if __name__ == "__main__":
# Replace this path with the actual path to your folder containing text files
    folder_path = 'C:/Users/Dell/'
    
# Replace 'Insights 47' with the specific text you want to remove
    text_to_remove = 'Insights 47'

    process_files_in_folder(folder_path, text_to_remove)


Processing file: C:/Users/Dell/blackcofferr/blackassign0001_Rising IT cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0002_Rising IT Cities and.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0003_Internet Demand's Ev.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0004_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0005_OTT platform and its.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0006_The rise of the OTT .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0007_Rise of Cyber Crime .txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0008_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0009_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0010_Rise of Cybercrime a.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0011_Rise of Internet Dem.txt
Processing file: C:/Users/Dell/blackcofferr/blackassign0012_Rise 

In [9]:
#this will save all column other than 'URL' and URL_ID' in Output New Data Structure

def remove_columns(input_excel_path, output_excel_path):

    input_wb = openpyxl.load_workbook(input_excel_path)

 
    output_wb = openpyxl.Workbook()
    output_sheet = output_wb.active

# Copy coulumns from input to output, excluding the first two columns
    for row in input_wb.active.iter_rows(min_row=1, values_only=True):
        output_sheet.append(row[2:])  # Skipping the first two columns


    output_wb.save(output_excel_path)

if __name__ == "__main__":

    input_excel_path = 'C:/Users/Dell/Output Data Structure.xlsx'

    output_excel_path = 'C:/Users/Dell/Output New Data Structure.xlsx'

    # Remove the first two columns and save the result
    remove_columns(input_excel_path, output_excel_path)


In [4]:
#this will save the two columns 'URL' and 'URL_ID' in Output filtered

workbook = openpyxl.load_workbook('Output Data Structure.xlsx')

sheet = workbook.active

# Keep the first two columns and drop the rest
for column in range(sheet.max_column, 2, -1):
    if column > 2:
        sheet.delete_cols(column)

# Save the modified Excel file
workbook.save('Output_filtered.xlsx')


In [ ]:
#finally this code will calculate all the variables that has been asked to measure

def analyze_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

 
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)


    positive_score = negative_score = polarity_score = subjectivity_score = 0
    avg_sentence_length = avg_words_per_sentence = complex_word_count = 0


    sia = SentimentIntensityAnalyzer()

    for sentence in sentences:
        blob = TextBlob(sentence)

        
        polarity_score += blob.sentiment.polarity
        subjectivity_score += blob.sentiment.subjectivity


        word_sentiments = [sia.polarity_scores(word)['compound'] for word in nltk.word_tokenize(sentence)]
        positive_score += sum(1 for score in word_sentiments if score > 0)
        negative_score += sum(1 for score in word_sentiments if score < 0)

   
        avg_sentence_length += len(sentence.split())


        complex_word_count += sum(1 for word in blob.words if syllable_count(word) > 2)


    num_sentences = len(sentences)
    avg_sentence_length /= num_sentences
    avg_words_per_sentence = len(words) / num_sentences
    complex_word_percentage = (complex_word_count / len(words)) * 100


    word_count = len(words)
    syllable_per_word = syllable_count(text) / word_count
    personal_pronouns = count_personal_pronouns(text)
    avg_word_length = sum(len(word) for word in words) / word_count

    return [
        positive_score,
        negative_score,
        polarity_score,
        subjectivity_score,
        avg_sentence_length,
        complex_word_percentage,
        fog_index(avg_sentence_length, complex_word_percentage),
        avg_words_per_sentence,
        complex_word_count,
        word_count,
        syllable_per_word,
        personal_pronouns,
        avg_word_length
    ]


def syllable_count(word):

    vowels = "aeiouy"
    count = 0
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    return count

def count_personal_pronouns(text):
 
    personal_pronouns_list = ["i", "me", "my", "mine", "we", "us", "our", "ours"]
    return sum(1 for word in nltk.word_tokenize(text.lower()) if word in personal_pronouns_list)

def fog_index(avg_sentence_length, complex_word_percentage):

    return 0.4 * (avg_sentence_length + complex_word_percentage)

def process_files_in_folder(folder_path, output_excel_path):

    workbook = openpyxl.Workbook()
    sheet = workbook.active


    output_structure_path = "C:/Users/Dell/Output New Data Structure.xlsx"
    
    output_structure_wb = openpyxl.load_workbook(output_structure_path)
    output_structure_wb = openpyxl.load_workbook(output_structure_path)
    output_structure_sheet = output_structure_wb.active

 
    variable_names = next(output_structure_sheet.iter_rows(min_row=1, max_row=1, values_only=True))
    sheet.append(variable_names)

    # Iterating through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")

            
            variable_values = analyze_text(file_path)

            
            sheet.append(variable_values)

    
    workbook.save(output_excel_path)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing text files
    folder_path = 'C:/Users/Dell/'

    # Replace 'output_results.xlsx' with the desired output Excel file path
    output_excel_path = 'output_results.xlsx'

    # Process files in the folder and save the results to the output Excel file
    process_files_in_folder(folder_path, output_excel_path)


In [38]:
df = pd.read_excel('C:/Users/Dell/Output_filtered.xlsx')
df1 = pd.read_excel('C:/Users/Dell/output_results.xlsx')

In [39]:
#just to concatenate two files 
result = pd.concat([df, df1], axis=1)

In [41]:
#finally converting the final file in excel
result.to_excel('output_result_finally.xlsx', index=False)
